In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
from implementation import *
DATA_TRAIN_PATH = 'train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

In [3]:
DATA_TEST_PATH = 'test.csv' # TODO: download train data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

## Do your thing crazy machine learning thing here :) ...

## Feature selection

In [27]:
ttx, missing_vect = change_data_missing(tX)
corr_mat = np.corrcoef(ttx.T)
columns = np.full((corr_mat.shape[0],), True, dtype = bool)

In [28]:
for i in range(corr_mat.shape[0]):
    for j in range(i+1,corr_mat.shape[1]):
        if np.abs(corr_mat[i][j])>0.9:
            if columns[j]:
                columns[j] = False
ttx = ttx[:,columns]
print(ttx.shape[1])

27


Now we will perform ridge regression on the new matrix

In [29]:
lambdas = np.logspace(-10,3,30)
degrees = np.arange(1,12,1)
err_fs = []
Weights_fs = []

In [30]:
for ind_d, degree in enumerate(degrees):
    phi_fs = build_poly_missing(ttx, missing_vect, degree)
    t_start = time.time()
    for ind_l, lambda_ in enumerate(lambdas):
        w_ri_fs, _ = ridge_regression(y, phi_fs, lambda_)
        error_fs = compute_accuracy(w_ri_fs, y, phi_fs)
        Weights_fs.append(w_ri_fs)
        err_fs.append(error_fs)
    best_err = np.min(err_fs[ind_d*len(lambdas):(ind_d+1)*len(lambdas)])
    argm = np.argmin(err_fs[ind_d*len(lambdas):(ind_d+1)*len(lambdas)])
    best_l = lambdas[argm]
    t = time.time() - t_start
    print("Elapsed time: {t} s, degree = {degree}, best error = {best_err}, best_lambda = {best_l}".format(t = t, degree = degree, best_err = best_err, best_l = best_l))

Elapsed time: 12.592736959457397 s, degree = 1, best error = 0.253328, best_lambda = 126.89610031679234
Elapsed time: 18.262277126312256 s, degree = 2, best error = 0.225688, best_lambda = 0.2592943797404667
Elapsed time: 31.20139765739441 s, degree = 3, best error = 0.214708, best_lambda = 2.04335971785694
Elapsed time: 84.47943997383118 s, degree = 4, best error = 0.20662, best_lambda = 0.09236708571873846
Elapsed time: 64.61517596244812 s, degree = 5, best error = 0.202912, best_lambda = 0.00018873918221350996
Elapsed time: 81.16721606254578 s, degree = 6, best error = 0.20038, best_lambda = 2.04335971785694
Elapsed time: 113.32259893417358 s, degree = 7, best error = 0.1952, best_lambda = 8.531678524172814e-06
Elapsed time: 139.87653303146362 s, degree = 8, best error = 0.187688, best_lambda = 0.2592943797404667
Elapsed time: 190.54938197135925 s, degree = 9, best error = 0.18316, best_lambda = 0.011721022975334793
Elapsed time: 202.40567827224731 s, degree = 10, best error = 0.188

## Choosing variables of order 2 randomly (pas vraiment utile apparemment)
We will use only degrees 8 to 11 as they give the bests results

In [27]:
degrees = np.arange(5,12,1)
lambdas = np.logspace(-10,0,30)
err = []
Weights_ord2 = []
ttx_ord2, missing_vect = change_data_missing(tX)

In [29]:
np.random.seed(1)
i, j = np.random.randint(0,30,2)
ttx = np.append(ttx_ord2, (ttx_ord2[:,i]*ttx_ord2[:,j]).reshape((ttx_ord2.shape[0],1)), axis = 1)
for ind_d, degree in enumerate(degrees):
    print('new degree')
    t_start = time.time()
    phi_ord2 = build_poly_missing(ttx, missing_vect, degree)
    for ind_l, lambda_ in enumerate(lambdas):
        w_ri_ord2, _ = ridge_regression(y, phi_ord2, lambda_)
        error_ord2 = compute_accuracy(w_ri_ord2, y, phi_ord2)
        Weights_ord2.append(w_ri_ord2)
        err.append(error_ord2)
    best_err = np.min(err)
    t = time.time() - t_start
    print("Elapsed time: {t} s, degree = {d}, best error = {best_err}, (i,j) = ({i},{j})".format(t = t, d = degree, best_err = best_err, i=i, j=j))

new degree
Elapsed time: 195.8911008834839 s, degree = 5, best error = 0.201032, (i,j) = (5,11)
new degree
Elapsed time: 226.64763593673706 s, degree = 6, best error = 0.199384, (i,j) = (5,11)
new degree
Elapsed time: 230.61801290512085 s, degree = 7, best error = 0.193856, (i,j) = (5,11)
new degree
Elapsed time: 340.2039473056793 s, degree = 8, best error = 0.186648, (i,j) = (5,11)
new degree
Elapsed time: 311.8979387283325 s, degree = 9, best error = 0.181924, (i,j) = (5,11)
new degree
Elapsed time: 495.0541481971741 s, degree = 10, best error = 0.181052, (i,j) = (5,11)
new degree
Elapsed time: 487.56017684936523 s, degree = 11, best error = 0.181052, (i,j) = (5,11)


## Ridge regression without feature selectionttx_ord2

In [15]:
from implementation import *
lambdas = np.logspace(-10,3,50)
degrees = np.arange(1,12,1)
err = []
Weights = []

In [16]:
tx_new, missing_vect = change_data_missing(tX)
for ind_d, degree in enumerate(degrees):
    phi = build_poly_missing(tx_new, missing_vect, degree)
    t_start = time.time()
    for ind_l, lambda_ in enumerate(lambdas):
        weights_ri, _ = ridge_regression(y, phi, lambda_)
        error = compute_accuracy(weights_ri, y, phi)
        Weights.append(weights_ri)
        err.append(error)
    best_err = np.min(err[ind_d*len(lambdas):(ind_d+1)*len(lambdas)])
    argm = np.argmin(err[ind_d*len(lambdas):(ind_d+1)*len(lambdas)])
    best_l = lambdas[argm]
    t = time.time() - t_start
    print("Elapsed time: {t} s, degree = {degree}, best error = {best_err}, best_lambda = {best_l}".format(t = t, degree = degree, best_err = best_err, best_l = best_l))

Elapsed time: 21.78075623512268 s, degree = 1, best error = 0.253088, best_lambda = 159.9858719606064
Elapsed time: 53.18994927406311 s, degree = 2, best error = 0.2252, best_lambda = 0.00023299518105153718
Elapsed time: 57.71281051635742 s, degree = 3, best error = 0.213452, best_lambda = 0.00042919342601287783
Elapsed time: 132.8130521774292 s, degree = 4, best error = 0.205316, best_lambda = 0.10481131341546875
Elapsed time: 137.60612559318542 s, degree = 5, best error = 0.201124, best_lambda = 1.2067926406393314
Elapsed time: 311.93634009361267 s, degree = 6, best error = 0.199452, best_lambda = 2.0235896477251596e-05
Elapsed time: 381.09544467926025 s, degree = 7, best error = 0.193916, best_lambda = 3.2374575428176467e-06
Elapsed time: 699.2371666431427 s, degree = 8, best error = 0.18674, best_lambda = 0.10481131341546875
Elapsed time: 582.3396110534668 s, degree = 9, best error = 0.181952, best_lambda = 6.250551925273976e-10
Elapsed time: 359.37212085723877 s, degree = 10, best

In [17]:
arg_err_min = np.argmin(err)
w_opt = Weights[arg_err_min]

In [19]:
tx_test_new, missing_vect_test = change_data_missing(tX_test)
phi_test = build_poly_missing(tx_test_new, missing_vect_test, degrees[arg_err_min // 50])

In [20]:
OUTPUT_PATH = 'Submission_ridge.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(w_opt, phi_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)

## Normal equations

In [10]:
from implementation import *
degrees2 = np.arange(15)
Weights2 = []
errors2 = []
tx2, missing_vect2 = change_data_missing(tX)
for iter_deg, degree in enumerate(degrees2):
    phi2 = build_poly_missing(tx2, missing_vect2, degree)
    w = np.linalg.solve(phi2.T @ phi2, phi2.T @ y)
    error = compute_accuracy(w,y,phi2)
    print("error ", iter_deg, ": ", error)
    Weights2.append(w)
    errors2.append(error)
m = np.argmin(errors2)
w_opt2 = Weights2[m]

error  0 :  0.342668
error  1 :  0.25318
error  2 :  0.225204
error  3 :  0.213456
error  4 :  0.205384
error  5 :  0.201132
error  6 :  0.19946
error  7 :  0.194336
error  8 :  0.186824
error  9 :  0.1821
error  10 :  0.18106
error  11 :  0.186968
error  12 :  0.223696


KeyboardInterrupt: 

In [ ]:
tx_test2, missing_vect_test2= change_data_missing(tX_test)
phi_test2 = build_poly(tx_test, missing_vect_test, degrees2[m])

In [ ]:
OUTPUT_PATH = 'submission_normal_eq.csv' # TODO: fill in desired name of output file for submission
y_pred2 = predict_labels(w_opt2, phi_test2)
create_csv_submission(ids_test, y_pred2, OUTPUT_PATH)

## Generate predictions and save ouput in csv format for submission:

In [15]:
tx_test, missing_test = change_data_missing(tX_test)
phi_test = build_poly_missing(tx_test, missing_test, degrees[m])

In [16]:
OUTPUT_PATH = 'submission_normal_eq_std.csv' # TODO: fill in desired name of output file for submission
y_pred_std = predict_labels(w_opt, phi_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)